In [1]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp
from config import consumer_key,consumer_secret,access_token,access_secret

In [2]:
gameid = 2021021242
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'
auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)

In [3]:
# time.sleep(1200) #20 minutes
# time.sleep(2400) #40 minutes
# time.sleep(3600) #1 hour
# time.sleep(5400) #1.5 hours
# time.sleep(7200) #2 hours
# time.sleep(9000) #2.5 hours
# time.sleep(10800) #3 hours
# time.sleep(24000) #6 hours 40 minutes
# time.sleep(28800)
# time.sleep(30000) #8 hours 20 minutes
# time.sleep(36000) #10 hours
# time.sleep(43200) #12 hours

# Use for delay start
pullurl = starturl + str(gameid) + endurl
pullurl
gameover = 'FALSE'
print(pullurl)

https://statsapi.web.nhl.com/api/v1/game/2021021242/feed/live.json


In [8]:
while gameid <= 2021021310:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    if home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings':
        print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow} -- {home_team} vs {away_team}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    period = 1
    while gameover == "FALSE" and obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings'):
        while gameover == 'FALSE':
            #print(x)
            #x=x+1
            pullurl = starturl + str(gameid) + endurl
            try:
                with urllib.request.urlopen(pullurl) as url:
                    obj = json.loads(url.read().decode())
            except:
                print(f'The URL is broke, bruh. Try again.')
            allPlays = obj["liveData"]["plays"]['allPlays']
            hometeamscore = 0
            awayteamscore = 0
            for play in allPlays:
                period = play['about']['ordinalNum']
                event = play['result']['event']
                timeee = play['about']['periodTimeRemaining']
                description = play['result']['description']
                if event == 'Goal':
                    if str(home_team) == 'Los Angeles Kings':
                        if play['team']['name'] == 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                    if str(home_team) != 'Los Angeles Kings':
                        if play['team']['name'] != 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                if f'Period: {period}-{timeee}, {event}-{description}' not in gamedetail:
                    gamedetail.append(f'Period: {period}-{timeee}, {event}-{description}')
                    if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
                        print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                        #Period Ready
                        #Giveaway
                        #Takeaway
                        #Hit
                    #elif event == 'Game Scheduled':
                        #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                        #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                        #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
                        #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
                        #time.sleep(8)
                    elif event == 'Period Start':
                        try:
                            api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                        #Period Start
                    elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                        try:
                            api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            if event == 'Game End':
                                gameover = 'TRUE'
                                gameused = gameid
                        except:
                            print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        time.sleep(8)
                        #Faceoff
                        #Blocked Shot
                        #Shot
                        #Penalty
                        #Missed Shot
                        #Stoppage
                        #Period Official
                        #Game End
                        #Game Official
                    elif event == 'Goal':
                        try:
                            api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                            print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        time.sleep(8)
                        #Goal
                    elif event == 'Period End' and (period == '1st' or period == '2nd'):
                        try:
                            api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        #time.sleep(1080)
                        #Period End
                    elif event == 'Period End' and (period == '3rd'):
                        try:
                            api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    elif event == 'Period End' and (period == 'OT'):
                        try:
                            api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    else:
                        time.sleep(0)
            print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')
    gameid += 1

2021021258-Live-2022-04-24-2022-04-24 -- Los Angeles Kings vs Anaheim Ducks
FAILED TO TWEET -- Period: 1st-00:00, Period Official-Period Official #GoKingsGo
NOT TWEETED---Period: 2nd-20:00, Period Ready-Period Ready
Period Starting: 2nd --- Score: Los Angeles Kings-1, Anaheim Ducks-0 #GoKingsGo
FAILED TO TWEET -- Period: 2nd-20:00, Faceoff-Anze Kopitar faceoff won against Ryan Getzlaf #GoKingsGo
NOT TWEETED---Period: 2nd-19:46, Hit-Anze Kopitar hit Ryan Getzlaf
FAILED TO TWEET -- Period: 2nd-18:45, Blocked Shot-Jamie Drysdale shot blocked shot by Jacob Moverare #GoKingsGo
FAILED TO TWEET -- Period: 2nd-18:39, Stoppage-Puck Frozen #GoKingsGo
FAILED TO TWEET -- Period: 2nd-18:39, Faceoff-Blake Lizotte faceoff won against Sam Steel #GoKingsGo
FAILED TO TWEET -- Period: 2nd-18:30, Blocked Shot-Alex Iafallo shot blocked shot by Jamie Drysdale #GoKingsGo
FAILED TO TWEET -- Period: 2nd-18:29, Stoppage-Puck in Netting #GoKingsGo
FAILED TO TWEET -- Period: 2nd-18:29, Faceoff-Blake Lizotte faceo

Period: 3rd-19:18, Shot-Andreas Athanasiou Backhand saved by Anthony Stolarz #GoKingsGo
Period: 3rd-18:52, Shot-Zach Aston-Reese Snap Shot saved by Jonathan Quick #GoKingsGo
Period: 3rd-18:51, Stoppage-Puck in Netting #GoKingsGo
Period: 3rd-18:51, Faceoff-Blake Lizotte faceoff won against Isac Lundestrom #GoKingsGo
Period: 3rd-18:40, Shot-Blake Lizotte Wrist Shot saved by Anthony Stolarz #GoKingsGo
Period: 3rd-17:59, Missed Shot-Arthur Kaliyev Wide of Net Anthony Stolarz #GoKingsGo
Los Angeles Kings-2, Anaheim Ducks-2
NOT TWEETED---Period: 3rd-18:03, Hit-Andrej Sustr hit Carl Grundstrom
Period: 3rd-17:37, Shot-Arthur Kaliyev Snap Shot saved by Anthony Stolarz #GoKingsGo
NOT TWEETED---Period: 3rd-17:04, Hit-Trevor Zegras hit Phillip Danault
Period: 3rd-16:46, Stoppage-Icing #GoKingsGo
Period: 3rd-16:46, Faceoff-Adam Henrique faceoff won against Phillip Danault #GoKingsGo
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los An

Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
L

Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
L

Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
Los Angeles Kings-2, Anaheim Ducks-2
L

...

Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
L

Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
L

Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
L

Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
L

Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
Los Angeles Kings-4, Anaheim Ducks-2
L

In [ ]:
#use to catch up to live
pullurl = starturl + str(gameid) + endurl
with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
allPlays = obj["liveData"]["plays"]['allPlays']
for play in allPlays:
    if f'Period: {period}-{timeee}, {event}' not in gamedetail:
        gamedetail.append(f'Period: {period}-{timeee}, {event}-{description}')
        print(play)

In [9]:
import json
import urllib.request
import pandas as pd

In [10]:
gameid = gameused
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'

In [11]:
pullurl = starturl + str(gameid) + endurl
gamelog = []
playerloggg = []
try:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    print(gameid)
    for play in allPlays:
        d4 = play['result']
        try:
            d4.update(play['about'])
        except:
            ok = 0
        try:
            d4.update(play["coordinates"])
        except:
            ok = 0
        try:
            for breaker in play['players']:
                d5 = breaker['player']
                d5.update(breaker)
                gamelog.append(d5)
        except:
            ok = 0
        try:
            d4.update(play["team"])
        except:
            ok = 0
        gamelog.append(d4)
        #print(d4)
#     for player in allPlays:
#         try:
#             for breaker in player['players']:
#                 d5 = breaker['player']
#                 d5.update(breaker)
#                 playerloggg.append(d5)
# #                 print(breaker['player'])
#         except:
#             ok = 1
except:
    print(f'Nope {gameid}')

2021021258


In [12]:
df1 = pd.DataFrame(gamelog)
df1

,event,eventCode,eventTypeId,description,eventIdx,eventId,period,periodType,ordinalNum,periodTime,...,y,name,triCode,secondaryType,seasonTotal,strength,gameWinningGoal,emptyNet,penaltySeverity,penaltyMinutes
0,Game Scheduled,LAK1,GAME_SCHEDULED,Game Scheduled,0.0,1.0,1.0,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Period Ready,LAK5,PERIOD_READY,Period Ready,1.0,5.0,1.0,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Period Start,LAK51,PERIOD_START,Period Start,2.0,51.0,1.0,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,Shot,LAK496,SHOT,Cam Fowler Slap Shot saved by Jonathan Quick,278.0,496.0,3.0,REGULAR,3rd,19:59,...,7.0,Anaheim Ducks,ANA,Slap Shot,NaN,NaN,NaN,NaN,NaN,NaN
733,Period End,LAK397,PERIOD_END,End of 3rd Period,279.0,397.0,3.0,REGULAR,3rd,20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,Period Official,LAK1001,PERIOD_OFFICIAL,Period Official,280.0,1001.0,3.0,REGULAR,3rd,20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df2 = pd.DataFrame(playerloggg)
df2

""


In [14]:
locationdf = df1[df1.x.notnull()]

In [15]:
locationdf.loc[(locationdf.x > 0), 'x']*=-1

/home/keoni/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [16]:
locationdf.loc[(locationdf.x > 0), 'y']*=-1

/home/keoni/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [17]:
# locationdf.to_csv("outputlesser.csv") 
locationdf = locationdf[locationdf["name"] == 'Los Angeles Kings']
locationdf = locationdf[locationdf["event"] != 'Faceoff']
locationdf = locationdf[locationdf["event"] != 'Hit']
locationdf = locationdf[locationdf["event"] != 'Giveaway']
locationdf = locationdf[locationdf["event"] != 'Takeaway']
locationdf = locationdf[locationdf["event"] != 'Penalty']
locationdf = locationdf[locationdf["event"] != 'Blocked Shot']

In [18]:
selectiondf = locationdf[["x","y","event","description","ordinalNum"]]
selectiondf

,x,y,event,description,ordinalNum
8,-59.0,-17.0,Missed Shot,Trevor Moore Wide of Net Anthony Stolarz,1st
29,-84.0,25.0,Missed Shot,Arthur Kaliyev Wide of Net Anthony Stolarz,1st
35,-61.0,-7.0,Missed Shot,Anze Kopitar Goalpost Anthony Stolarz,1st
56,-61.0,24.0,Shot,Blake Lizotte Snap Shot saved by Anthony Stolarz,1st
80,-75.0,1.0,Shot,Carl Grundstrom Backhand saved by Anthony Stolarz,1st
89,-53.0,37.0,Missed Shot,Adrian Kempe Wide of Net Anthony Stolarz,1st
92,-58.0,-3.0,Shot,Matt Roy Snap Shot saved by Anthony Stolarz,1st
95,-76.0,3.0,Shot,Anze Kopitar Backhand saved by Anthony Stolarz,1st
117,-68.0,39.0,Shot,Carl Grundstrom Wrist Shot saved by Anthony St...,1st
122,-77.0,10.0,Goal,"Arthur Kaliyev (14) Snap Shot, assists: Quinto...",1st


In [19]:
selectiondf.to_csv("Kings.csv",index=False) 

In [20]:
locationdf = df1[df1.x.notnull()]

In [21]:
locationdf.loc[(locationdf.x < 0), 'x']*=-1

/home/keoni/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [22]:
locationdf.loc[(locationdf.x < 0), 'y']*=-1

/home/keoni/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [23]:
# locationdf.to_csv("outputlesser.csv") 
locationdf = locationdf[locationdf["name"] != 'Los Angeles Kings']
# locationdf = locationdf[locationdf["name"] != '']
locationdf = locationdf[locationdf["event"] != 'Faceoff']
locationdf = locationdf[locationdf["event"] != 'Hit']
locationdf = locationdf[locationdf["event"] != 'Giveaway']
locationdf = locationdf[locationdf["event"] != 'Takeaway']
locationdf = locationdf[locationdf["event"] != 'Penalty']
locationdf = locationdf[locationdf["event"] != 'Blocked Shot']

In [24]:
selectiondf = locationdf[["x","y","event","description","ordinalNum"]]
selectiondf

,x,y,event,description,ordinalNum
53,53.0,-15.0,Shot,Troy Terry Wrist Shot saved by Jonathan Quick,1st
70,64.0,8.0,Missed Shot,Sam Steel Wide of Net Jonathan Quick,1st
98,59.0,26.0,Shot,Josh Mahura Wrist Shot saved by Jonathan Quick,1st
109,36.0,-29.0,Shot,Jamie Drysdale Slap Shot saved by Jonathan Quick,1st
139,77.0,-5.0,Shot,Ryan Getzlaf Backhand saved by Jonathan Quick,1st
142,77.0,6.0,Shot,Troy Terry Wrist Shot saved by Jonathan Quick,1st
157,77.0,2.0,Shot,Isac Lundestrom Wrist Shot saved by Jonathan Q...,1st
160,79.0,5.0,Shot,Zach Aston-Reese Wrist Shot saved by Jonathan ...,1st
197,75.0,-6.0,Shot,Dominik Simon Snap Shot saved by Jonathan Quick,1st
203,81.0,-17.0,Missed Shot,Dominik Simon Wide of Net Jonathan Quick,1st


In [25]:
selectiondf.to_csv("Opponent.csv",index=False) 